<h2>TASK 1: A Python Problem</h2>

</h5><b>Instructions</b></h5>

Make sure your codes are properly documented. We recommend the following:-
Before each code block have a markdown block which mentions the following
- What the code block is doing.
- What is your intuition behind doing this? Why do you think it is useful?
- For bigger code blocks also add comments in the block.
- Keep everything in this notebook things which worked, things which did not work.
- This notebook should be a snapshot of the process you follow to solve a Data
  problem.

<hr />

## PROBLEM
##### You have to write a python script which can fetch all the tweets (as many as allowed by Twitter API) done by midas@IIITD twitter handle and dump the responses into JSONlines file.
##### The other part of your script should be able to parse these JSONline files to display the following for every tweet in a tabular format.
- The text of the tweet.
- Date and time of the tweet.
- The number of favorites/likes.
- The number of retweets.
- Number of Images present in Tweet. If no image returns None.

## Solution


<p style="color:red">Note: As it is not mentioned in the question to remove links or hashtags from the tweet text, I have not performed that task. However it can be easily remove by regular expressions or by tokenizing the tweet text and removing individually.</p>

<p style="color:red">Note: In part 2 of task one, I have stored the required data from tweet in an excel file</p>

<p style="color:red">Note: In part 1 of task one, tweet is appended one by one. Running the same block will again append the tweets at the end. That is why i have checked whether the file exists in the directory before appending.</p>

<p style="color:red">Note: In part 2 of task one, Date and time has been kept together. To separate them, we can use <br /> [date = datetime.datetime.strptime(string, "%d %b %Y  %H:%M:%S.%f")] <br />
and access year,time etc. by date.year, date.time etc.</p>




### Part 1

In [58]:
# Importing the necessary libraries for completing the task

import json
import jsonlines as jsl
import tweepy
import datetime
import xlsxwriter
import os

<H4>Step 1: Initialize access and consumer variables </H4>

In [59]:
ACCESS_TOKEN = '1114284772665126912-yIWbS51Mh0Xri4Hs1keqRBLMmorQMX'
ACCESS_SECRET = 'WpLwxRQuiATZT520FoJBR1YpesuBucasTJ4yOuywYJ4cL'
CONSUMER_KEY = 'C6jm3wI4AttxBBLwPHeY3LfiE'
CONSUMER_SECRET = 'xoFnaDnn4yib6IGNWZIFHCtt4P8ZKehaqQikbORZObtg0dvKAE'

<h4>Step 2: Setup tweepy to access the API</h4>

In [60]:
# setup tweepy by authenticating the user credentials
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, compression=True)

<h4>Step 3: Get twitter data from midasiiitd timeline and store them in a list</h4>

In [61]:
USER = "midasiiitd"

tweets_list = []

# mode = extended because tweets with longer text are truncated  
# get tweet one by one and append in list

for status in tweepy.Cursor(api.user_timeline, id=USER,tweet_mode="extended").items(): 
    tweets_list.append(status)

<h4> Step 4: Dump the tweets in a jsonl file </h4>

In [68]:
path = 'final.jsonl' # name of the file where tweets are stored

# first check if file already present
exists = os.path.isfile(path)
if exists==False:

    # To store them in jsonl format - tweet is appended one by one followed by a linebreak
    for i in range(0,len(tweets_list)):
        with open(path, 'a', encoding='utf8') as file: 
            json.dump(tweets_list[i]._json,file)
            file.write('\n')
else:
    print("File already exists. Delete it first.")

File already exists. Delete it first.


### Second Part

<h4> Step 5: Function to get required data from a tweet </h4>

<ul>
    <li> Likes/favourites: Present in 'favourite_count'</li>
    <li> Retweet count: Present in 'retweet_count'</li>
    <li> Number of image: Check for 'media' in 'entities'. If found, check the file format of each item in 'media'</li>
    <li> Date and time: Present in 'created_at'</li>
    <li> Tweet text: Present in 'full_text'</li>
</ul>

In [63]:
# Common image formats
img_fmt = ['png','jpeg','jpg',"gif","bmp"]

# function to extract data from 'tweet'
def get_data(tweet):
    favourite_count = tweet['favorite_count']
    retweet_count = tweet['retweet_count']
    number_image=0
    if ('media' not in tweet['entities']):
        number_image = None
    else:
        for media in tweet['entities']['media']:
            if media['media_url'][-3:] in img_fmt:
                number_image+=1
    if ("retweeted_status" in tweet):
        text = tweet['retweeted_status']['full_text']
    else:
        text = tweet['full_text']
    return [text,tweet['created_at'],favourite_count,retweet_count,number_image]

<h4> Step 6: Reading tweet line by line from jsonl file and storing in an excel file</h4>

In [67]:
# data to be obtain
arr=["text","date and time","likes count","retweet count","image count"]

#create a workbook and sheet
workbook = xlsxwriter.Workbook('final.xlsx') 
worksheet = workbook.add_worksheet() 

# add column headers from arr
for i in range(0,len(arr)):
    worksheet.write(0, i, arr[i]) 

# Each line in .jsonl file is a tweet
# Use the function 'get_data' to extract info.

row=1 
for line in open(path):
    tweet = json.loads(line)
    l= get_data(tweet)
    for i in range(0,len(l)):
        worksheet.write(row, i, l[i]) 
    row+=1

workbook.close()

/Users/abhinav/dev/newcvtest/lib/python3.6/site-packages/xlsxwriter/worksheet.py:909: UserWarning: Ignoring URL 'https://t.co/46MisEZT5n%20Deep%20learning%20for%20predicting%20aftershocks%20of%20large%20earthquakes.%20Besides%20offering%20better%20predictions,%20interpretations%20of%20the%20model%20suggest%20promising%20directions%20for%20new%20physical%20theories%20https://t.co/b4bAvo6YZG' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/Users/abhinav/dev/newcvtest/lib/python3.6/site-packages/xlsxwriter/worksheet.py:909: UserWarning: Ignoring URL 'https://t.co/hYiWI7ntyk%20TensorFuzz%20automates%20the%20process%20of%20finding%20inputs%20that%20cause%20some%20specific%20testable%20behavior,%20like%20disagreement%20between%20float16%20and%20float32%20implementations%20of%20a%20neural%20network%20https://t.co/Nt9YX5kJXU' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
